In [13]:
import numpy as np
import sys
sys.path.append ('../src')
from NeuralNetwork_v2 import SigmoidF,ReluF,TanhF,LReluF,Activation_V2,Linear,Layer,NNClassifier_V2,SoftmaxF, BinaryCrossEntropy, CategoricalCrossEntropy
# Load the dataset
import numpy as np
import h5py
import sys
sys.path.append ('../src')
train_dataset = h5py.File ('Data/train_catvnoncat.h5','r')
X_train_orig = np.array (train_dataset['train_set_x'][:])
Y_train_orig = np.array (train_dataset['train_set_y'][:])
test_dataset = h5py.File ('Data/test_catvnoncat.h5','r')
X_test_orig = np.array (test_dataset['test_set_x'][:])
Y_test_orig = np.array (test_dataset['test_set_y'][:])

X_train = X_train_orig.reshape (X_train_orig.shape[0],-1) / 255
Y_train = np.array (Y_train_orig).reshape (Y_train_orig.shape[0],1)
X_test = X_test_orig.reshape (X_test_orig.shape[0],-1) / 255
Y_test = np.array (Y_test_orig).reshape (Y_test_orig.shape[0],1)

X_train = X_train.T
Y_train = Y_train.T
X_test = X_test.T
Y_test = Y_test.T


In [14]:
print (X_train.shape)

(12288, 209)


In [15]:
activation = Activation_V2 (function=LReluF)

In [16]:
pred = activation.forward (np.array([0,2,-0.5]))

In [17]:
dZ = activation.backward(pred)

In [18]:
np.random.seed(2)

dA = np.random.randn(1,2)
A = np.random.randn(3,2)
W = np.random.randn(1,3)
b = np.random.randn(1,1)
Z = np.random.randn(1,2)

linear = Linear (1,3)
Zolv = linear.forward (A,W,b)
print (Zolv.shape)
activation = Activation_V2 (function=SigmoidF)
A = activation.forward (Z)
dZ = activation.backward (dA)
dA_prev,dW,db = linear.backward (dZ)
np.testing.assert_array_almost_equal(dA_prev, [[0.11018,  0.01105], [ 0.09467,  0.00949], [-0.05743, -0.00576]], decimal=5)
np.testing.assert_approx_equal(db, -0.057296, significant=5)
np.testing.assert_array_almost_equal(dW, [[ 0.10267,  0.09778, -0.01968]] , decimal=5)

activation = Activation_V2 (function=ReluF)
A = activation.forward (Z)
dZ = activation.backward (dA)
dA_prev,dW,db = linear.backward (dZ)
np.testing.assert_array_almost_equal(dA_prev, [[ 0.4409,  0.], [ 0.37884,  0. ], [-0.22982, 0. ]] , decimal=5)
np.testing.assert_array_almost_equal(dW, [[ 0.44514,  0.37371, -0.10479]] , decimal=5)
np.testing.assert_approx_equal(db, -0.20838, significant=5)

(1, 2)


In [19]:
np.random.seed(2)

dA = np.random.randn(1,2)
A = np.random.randn(3,2)
W = np.random.randn(1,3)
b = np.random.randn(1,1)
Z = np.random.randn(1,2)

layer = Layer (1,3,activationF=SigmoidF)
layer.W = W
layer.b = b
pred = layer.forward (A)
dA_prev,dW,db= layer.backward (dA)

In [20]:
layer1 = Layer (4,5,activationF=ReluF)
layer2 = Layer (3,4,activationF=ReluF)
layer3 = Layer (1,3,activationF=SigmoidF)

np.random.seed(6)
X = np.random.randn(5,4)
W1 = np.random.randn(4,5)
b1 = np.random.randn(4,1)
W2 = np.random.randn(3,4)
b2 = np.random.randn(3,1)
W3 = np.random.randn(1,3)
b3 = np.random.randn(1,1)

layer1.W = W1
layer1.b = b1
layer2.W = W2
layer2.b = b2
layer3.W = W3
layer3.b = b3

A1 = layer1.forward (X)
A2 = layer2.forward (A1)
A3 = layer3.forward (A2)

np.testing.assert_array_almost_equal(A3, [[0.03922,  0.704989,  0.19734,  0.04728]], decimal=5)


In [21]:
layer1 = Layer (4,5,activationF=ReluF)
layer2 = Layer (3,4,activationF=ReluF)
layer3 = Layer (1,3,activationF=SigmoidF)
layers = [layer1,layer2,layer3]
y = np.asarray([[1, 1, 1]])
al = np.array([[.8,.9,0.4]])
nn = NNClassifier_V2 (layers)
cost = nn.costFunction (y.T,al.T)
np.testing.assert_approx_equal(cost, 0.41493, significant=5)
print (cost)

[[0.4149316]]


In [22]:
layer1 = Layer (3,4,activationF=ReluF)
layer2 = Layer (1,3,activationF=SigmoidF)
layers = [layer1,layer2]

np.random.seed(3)
AL = np.random.randn(1, 2)
Y = np.array([[1, 0]])
A1 = np.random.randn(4,2)  #X
W1 = np.random.randn(3,4)
b1 = np.random.randn(3,1)
Z1 = np.random.randn(3,2)  # cache activation
A2 = np.random.randn(3,2)
W2 = np.random.randn(1,3)
b2 = np.random.randn(1,1)
Z2 = np.random.randn(1,2) # cache activation

layer1.W = W1
layer1.b = b1
layer2.W = W2
layer2.b = b2

nn = NNClassifier_V2 (layers)
nn.forward (A1)          # dejamos en cache A,W,b,Z
print (layer1.linear.cache.keys())
print (layer1.activation.cache.keys())
print (layer2.linear.cache.keys())
print (layer2.activation.cache.keys())

layer1.activation.cache ['Z'] = Z1
layer1.linear.cache['A'] = A1
layer2.activation.cache ['Z'] = Z2
layer2.linear.cache['A'] = A2

nn.backward (AL,Y)

np.testing.assert_array_almost_equal(layer1.cache["dW"], [[ 0.4101, 0.078072, 0.13798, 0.10502], [ 0., 0., 0., 0. ], [0.05284, 0.01006, 0.01778, 0.01353]] , decimal=5)
np.testing.assert_array_almost_equal(layer1.cache["db"], [[-0.22007], [ 0.], [-0.02835]], decimal=5)
np.testing.assert_array_almost_equal(layer2.cache["dA_prev"], [[ 0.12913, -0.44014], [-0.14176,0.48317], [0.01664, -0.05671]], decimal=5)

print (layer1.cache.keys())
print (layer2.cache.keys())

dict_keys(['A', 'W', 'b'])
dict_keys(['Z'])
dict_keys(['A', 'W', 'b'])
dict_keys(['Z'])
dict_keys(['dA_prev', 'dW', 'db'])
dict_keys(['dA_prev', 'dW', 'db'])


In [24]:
np.random.seed(1)
layer1 = Layer (20,12288,activationF=ReluF,keep_prob=1.0)
layer2 = Layer (7,20,activationF=ReluF)
layer3 = Layer (5,7,activationF=ReluF)
layer4 = Layer (1,5,activationF=SigmoidF)
layers = [layer1,layer2,layer3,layer4]
nn = NNClassifier_V2 (layers,costFunction=BinaryCrossEntropy,kargs={'steps':3000,'learning_rate':0.0075,'mini_batch_size':2**8},print_cost=True)
costs=nn.optimize (X_train,Y_train,l2_lambda = 0.7,method='SGD')

LR = 0.0075:   0%|                                                                    | 3/3000 [00:00<02:06, 23.69it/s]

Cost after iteration 0: 0.771749


LR = 0.0075:   4%|██▎                                                               | 105/3000 [00:04<01:53, 25.45it/s]

Cost after iteration 100: 0.672053


LR = 0.0075:   7%|████▍                                                             | 204/3000 [00:08<02:12, 21.03it/s]

...

LR = 0.0075:  94%|████████████████████████████████████████████████████████████▊    | 2805/3000 [02:03<00:08, 22.70it/s]

Cost after iteration 2800: 0.078247


LR = 0.0075:  97%|██████████████████████████████████████████████████████████████▉  | 2904/3000 [02:08<00:04, 22.70it/s]

Cost after iteration 2900: 0.075444


LR = 0.0075: 100%|█████████████████████████████████████████████████████████████████| 3000/3000 [02:12<00:00, 22.64it/s]


In [25]:
print (costs[-1])

[[0.07298434]]


In [26]:
train_prediction = nn.forward (X_train)
train_prediction[train_prediction >= 0.5] = 1
train_prediction[train_prediction < 0.5] = 0
accuracy = np.mean(Y_train==train_prediction) * 100
print ('Trainig set accuracy :' , accuracy  )

test_prediction = nn.forward (X_test)
test_prediction[test_prediction >= 0.5] = 1
test_prediction[test_prediction < 0.5] = 0
accuracy = np.mean(Y_test==test_prediction) * 100
print ('Trainig set accuracy :' , accuracy  )


Trainig set accuracy : 99.04306220095694
Trainig set accuracy : 82.0
